# Relevant Collection Interfaces

C# code in real applications benefits from **programming to interfaces**. This principle means you depend on an abstraction (an interface) rather than on a concrete class. By coding against contracts like `IEnumerable<T>` or `ICollection<T>`, your methods remain flexible: callers can pass in arrays, lists, queries, or any other type that implements the interface. This reduces coupling, makes your code more testable, and allows implementations to change without breaking your API. Two of the most relevant collection interfaces are `IEnumerable<T>` and `ICollection<T>`. Knowing when to use each keeps your APIs clear, efficient, and maintainable.

## What they represent

- **`IEnumerable<T>`**: the minimum contract for *iteration*. You can `foreach` it and use LINQ. No guarantees about count or mutability.
- **`ICollection<T>`**: builds on `IEnumerable<T>` and adds *size* and *mutation*. It exposes `Count`, `Add`, `Remove`, `Contains`, `Clear`, and `IsReadOnly`.

## Rule of thumb

- **Accept** `IEnumerable<T>` for inputs you will *read* or *iterate* over.
- **Return** `IReadOnlyCollection<T>` when you want to expose a *countable, read‑only* collection.
- **Use** `ICollection<T>` *inside* aggregates or services when you must *mutate* the collection.

## Practical comparisons

**Capabilities at a glance**

| Interface | Iterate | Count | Add/Remove | Intent |
| --- | --- | --- | --- | --- |
| `IEnumerable<T>` | ✓ | ✗ (you may have to iterate to count) | ✗ | Read/stream items |
| `ICollection<T>` | ✓ | ✓ (`Count`) | ✓ | Manage a mutable collection |
| `IReadOnlyCollection<T>` | ✓ | ✓ (`Count`) | ✗ | Expose a stable, read‑only view |

## Real‑world usage patterns

### 1) Service accepting input data

In [ ]:
// Program.cs
InvoiceService service = new InvoiceService();

var lines = new List<InvoiceService.InvoiceLine>
{
    new InvoiceService.InvoiceLine { UnitPrice = 10.0m, Quantity = 2 },
    new InvoiceService.InvoiceLine { UnitPrice = 5.0m, Quantity = 5 }
};

decimal total = service.CalculateTotal(lines);
Console.WriteLine($"Total Invoice Amount: {total}");

// InvoiceService.cs
public class InvoiceService
{
    // We only need to iterate; callers can pass arrays, lists, EF queries, etc.
    public class InvoiceLine
    {
        public decimal UnitPrice { get; set; }
        public int Quantity { get; set; }
    }

    public decimal CalculateTotal(IEnumerable<InvoiceLine> lines)
    {
        decimal total = 0m;
        foreach (var line in lines)
        {
            total += line.UnitPrice * line.Quantity;
        }
        return total;
    }
}

Why `IEnumerable<T>`: the service doesn't mutate or need `Count`; it just streams through items.

### 2) Aggregate that manages internal state

In [ ]:
// Program.cs
ShoppingBasket basket = new();

basket.AddItem(new BasketItem { ProductId = Guid.NewGuid(), ProductName = "Apple", Price = 0.5m, Quantity = 3 });
basket.AddItem(new BasketItem { ProductId = Guid.NewGuid(), ProductName = "Banana", Price = 0.2m, Quantity = 5 });

Console.WriteLine("Items in basket: " + basket.Items.Count);
// Console.WriteLine("Items in basket: " + basket._items); // This line will cause a compile-time error

// BasketItem.cs
public class BasketItem
{
    public Guid ProductId { get; set; }
    public string ProductName { get; set; } = string.Empty;
    public decimal Price { get; set; }
    public int Quantity { get; set; }
}

// ShoppingBasket.cs
public class ShoppingBasket
{
    private ICollection<BasketItem> _items = new List<BasketItem>();

    public IReadOnlyCollection<BasketItem> Items => (IReadOnlyCollection<BasketItem>)_items;

    public void AddItem(BasketItem item)
    {
        // business rules here
        _items.Add(item);
    }

    public bool RemoveItem(Guid productId)
    {
        BasketItem? toRemove = null;
        foreach (var i in _items)
        {
            if (i.ProductId == productId) { toRemove = i; break; }
        }
        return toRemove != null && _items.Remove(toRemove);
    }
}

Why `ICollection<T>` internally: the basket *mutates*. Why `IReadOnlyCollection<T>` externally: consumers can read and count, but not modify.

### 3) API surface returning read‑only data

In [ ]:
// Program.cs
CourseCatalog catalog = new();

foreach (var course in catalog.ListCourses())
{
    Console.WriteLine($"{course.Title} by {course.Instructor}");
}

// Course.cs
public class Course
{
    public required string Title { get; set; }
    public required string Instructor { get; set; }
}

// ICourseCatalog.cs
public interface ICourseCatalog
{
    IReadOnlyCollection<Course> ListCourses();
}

// CourseCatalog.cs
public class CourseCatalog : ICourseCatalog
{
    private readonly List<Course> _courses = new()
    {
        new() { Title = "C# Basics", Instructor = "John Doe" },
        new() { Title = "Advanced C#", Instructor = "Jane Smith" }
    };

    public IReadOnlyCollection<Course> ListCourses()
    {
        return _courses.AsReadOnly();
    }
}

Why `IReadOnlyCollection<T>`: callers can iterate and know the `Count` without being able to change the returned collection. Here we are creating a custom interface that our `CourseCatalog` class implements.

## IEnumerable and lazy vs eager evaluation

`IEnumerable<T>` can be *more performant for iteration* because it supports **lazy evaluation**. Lazy evaluation means elements are produced **on demand** as you iterate, instead of being computed and stored up front.

### Lazy evaluation example

In [ ]:
static IEnumerable<int> GenerateNumbersLazy(int count)
{
    for (int i = 0; i < count; i++)
    {
        Console.WriteLine($"[Lazy] Generating {i}");
        yield return i; // generated on demand
    }
}

Console.WriteLine("=== Using Lazy (IEnumerable) ===");
var lazy = GenerateNumbersLazy(10); // nothing executed yet
foreach (var x in lazy.Take(3)) // only generates 3 numbers
{
    Console.WriteLine($"Using {x}");
}

Here, the iterator method with `yield return` produces values only as the `foreach` asks for them.

### Eager evaluation example (materialise first)

In [ ]:
static List<int> GenerateNumbersEager(int count)
{
    var list = new List<int>();
    for (int i = 0; i < count; i++)
    {
        Console.WriteLine($"[Eager] Generating {i}");
        list.Add(i); // everything computed right now
    }
    return list;
}

Console.WriteLine("=== Using Eager (List) ===");
var eager = GenerateNumbersEager(10); // generates ALL 10 immediately
foreach (var x in eager.Take(3)) // still only use 3
{
    Console.WriteLine($"Using {x}");
}

Eager materialisation computes and stores every value immediately. This can be wasteful when you only need a small slice.

Lazy sequences shine for large (or potentially unbounded) data and when you expect to consume only part of the sequence. Try messing with the number passed to `GenerateNumbersEager` and `GenerateNumbersLazy` to understand why lazy evaluation is preferable!

## Common pitfalls and tips

- **Do not** expose `ICollection<T>` publicly unless callers are meant to mutate it.
- **Do** prefer `IEnumerable<T>` for parameters; it maximises flexibility for callers.
- If your method needs `Count` and nothing else, prefer `IReadOnlyCollection<T>` to signal that you will not mutate.
- Be mindful of **deferred execution** with iterator methods and many sequence providers; materialise to a concrete list when you need a stable snapshot or multiple passes.

## Mini checklist

- Reading only → **`IEnumerable<T>` parameter**.
- Need `Count` but no mutation → **`IReadOnlyCollection<T>` return type**.
- Managing items internally → **`ICollection<T>` field** plus read‑only exposure.